In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
import pickle
import numpy as np

In [2]:
# Building AlexNet

class AlexNet():
    
    def __init__(self):
        self.model = Sequential()
        self.model.add(Conv2D(96, input_shape=(640, 640, 3), kernel_size=11, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=5, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(384, kernel_size=3, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=3, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Flatten())
        self.model.add(Dense(4096, activation="leaky_relu"))
        self.model.add(Dense(4096, activation="linear"))
        self.model.add(Dense(32))
        self.model.compile(optimizer="sgd", loss="mean_squared_error", metrics="accuracy")

In [3]:
model = AlexNet()
#model.model.summary()

In [4]:
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [5]:
# Create custom dataset

with open('data_annote', 'rb') as f:
    DATA = pickle.load(f)
    
file_paths = [idx[0] for idx in DATA]
joint_keyframes = np.asarray([np.asarray(idx[1]).flatten() for idx in DATA])

ds_train = tf.data.Dataset.from_tensor_slices((file_paths, joint_keyframes))

def read_image(image_file, joint_keyframes):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_image(image, channels=3, dtype=tf.float32)
    image.set_shape([640, 640, 3])
    image = tf.image.resize(image, [640, 640])
    return image, joint_keyframes

def image_resize(image_file, joint_keyframes):
    image = tf.io.read_file(image_file)
    image =  tf.image.decode_jpeg(image, channels=3, dtype=tf.float32) 
    image_file = tf.image.resize(image, [224,224,3])
    
    return image_file, joint_keyframes


ds_train = ds_train.map(read_image).batch(2)

In [ ]:
model.model.fit(ds_train, epochs=10)

Epoch 1/10
   5/7340 [..............................] - ETA: 10:12:39 - loss: nan - accuracy: 0.0000e+00          

In [ ]:
joint_keyframes[100].shape